## Table of Contents
1. [TF-IDF Split](#tfidf)
2. [Baseline Model](#base)
3. [Initial Models](#initial)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from custom import * #personal functions
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_pickle('../Data/classify_reviews.pkl')

In [3]:
df

,Review,Score
0,\n Keough's...,1
1,\n While Th...,1
2,"\n If ""The ...",1
3,"\n ""The Lod...",1
4,\n There's ...,0
...,...,...
9169,\n Resoluti...,1
9170,\n Ambiguit...,1
9171,\n The movi...,1
9172,"\n a slick,...",1


# TF-IDF Split <a id='tfidf'></a>

In [4]:
vectorizer = TfidfVectorizer()

tf_idf_data = vectorizer.fit_transform(df['Review'])

In [5]:
print(tf_idf_data[0])

  (0, 10117)	0.2914117875380376
  (0, 10966)	0.22312090826351919
  (0, 754)	0.16094520332915854
  (0, 7775)	0.1309246803411478
  (0, 1724)	0.2914117875380376
  (0, 6811)	0.1276353341610168
  (0, 16518)	0.16715932414848764
  (0, 6425)	0.268295219747871
  (0, 10010)	0.11837660664392648
  (0, 5403)	0.24963194496140165
  (0, 15392)	0.24733102181622205
  (0, 8391)	0.27274851275156836
  (0, 16188)	0.30754240978603853
  (0, 14952)	0.14632933226399633
  (0, 14784)	0.1432134391398423
  (0, 2020)	0.11907021045199254
  (0, 14783)	0.08126429283884655
  (0, 4642)	0.171213209291973
  (0, 14147)	0.19595693675204542
  (0, 13522)	0.2576463758521595
  (0, 7871)	0.07055357563790901
  (0, 16492)	0.16715932414848764
  (0, 8120)	0.2914117875380376


In [6]:
tf_idf_data.shape

(9174, 16679)

Our vectorized data contains 9,244 reviews, with 16,791 unique words in the vocabulary.



# Baseline Model  <a id='base'></a>

First, let's make a dummy model which always predicts the majority class. We'll use this as a comparison for all of our actual models.

In [7]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(tf_idf_data, df['Score'], random_state=13)

In [8]:
# Create dummy classifer
dummy = DummyClassifier(strategy='most_frequent', random_state=1)

# "Train" model
dummy.fit(X_train, y_train)

DummyClassifier(random_state=1, strategy='most_frequent')

In [9]:
summary_df = pd.DataFrame(columns=['Model','Accuracy', 'Recall', 'Precision',  'F1'])
predictions = dummy.predict(X_test)
summary_df = summary_df.append({'Model': 'Dummy',
   'Accuracy': metrics.accuracy_score(y_test, predictions),
   'Recall': metrics.recall_score(y_test, predictions),
   'Precision': metrics.precision_score(y_test, predictions), 
   'F1': metrics.f1_score(y_test, predictions)},ignore_index=True)
summary_df

,Model,Accuracy,Recall,Precision,F1
0,Dummy,0.563208,1.0,0.563208,0.72058


In [10]:
print(f"The majority class is {predictions[0]}.")

The majority class is 1.


Here, we can see the dummy model is predicting only class "1". This gives us a recall of 1 (obviously), and an accuracy and precision of 0.55. Let's hope our models will need to do better than this!

# Initial Models <a id='initial'></a>

Using my classification_models() function, let's create models with only the default parameters.

In [28]:
models = ['logistic','knn','tree','rf','AdaBoost','xgb','GrdBoost','svc','Bayes']
classification_models(tf_idf_data,df['Score'],models)

Using logistic
Using knn
Using tree
Using rf
Using AdaBoost
Using xgb
Using GrdBoost
Using svc
Using Bayes


,Model,Accuracy,Recall,Precision,F1
0,logistic,0.756625,0.825919,0.759786,0.791474
1,knn,0.492158,0.302708,0.589454,0.400000
2,tree,0.646295,0.672147,0.688119,0.680039
3,rf,0.718226,0.828820,0.713572,0.766890
4,AdaBoost,0.624121,0.807544,0.627348,0.706131
5,xgb,0.635479,0.844294,0.629870,0.721488
6,GrdBoost,0.654408,0.863636,0.641984,0.736495
7,svc,0.770146,0.837524,0.771149,0.802967
8,Bayes,0.728502,0.911025,0.696746,0.789606


We want to prioritize accuracy in our text classification, and the best of these models are SVC, Logistic Regression, Multinomial Naive Bayes. Let's check to see if using gridsearch gets us any better results.

In [13]:
grid_models = ['logistic','svc','Bayes']
param_grid = [{'C': np.logspace(-4, 4, 20),
              'fit_intercept': [True, False],
              'max_iter': [1_000,10_000],
              'warm_start': [False,True]}, #logistic
              {'C' : np.linspace(0.01,10,6),
              'kernel': ['rbf', 'poly', 'sigmoid'],
              'degree': [3,2],
              'gamma': ['scale','auto'],
              'shrinking': [True, False]}, #svc
              {'alpha': [1,0.5],
              'fit_prior': [True, False]}] #bayes

In [14]:
classification_models(tf_idf_data,df['Score'],grid_models,param_grid=param_grid,grid=True)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 800 out of 800 | elapsed:  2.9min finished


logistic's best parameters are {'C': 11.288378916846883, 'fit_intercept': True, 'max_iter': 1000, 'warm_start': False}
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 115.7min finished


svc's best parameters are {'C': 4.006, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly', 'shrinking': True}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Bayes's best parameters are {'alpha': 0.5, 'fit_prior': False}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished


,Model,Accuracy,Recall,Precision,F1
0,logistic,0.759128,0.785783,0.788814,0.787295
1,svc,0.778202,0.830932,0.789234,0.809546
2,Bayes,0.771117,0.800192,0.797129,0.798658


In [26]:
grid_models = ['knn','tree','rf','AdaBoost','GrdBoost','svc']
param_grid = [#{'penalty': ['l1', 'l2'], 
               #'C': np.logspace(-4, 4, 10)}, #logistic
              {'n_neighbors': [1,3,5],
               'weights':['uniform','distance'],
               'metric': ['euclidean','manhattan']}, #knn
              {'criterion': ['gini', 'entropy'],
               'max_depth' : [2,5,50],
               'min_samples_leaf':[1,2,8],
               'min_samples_split':[2,4]}, #tree
              {'n_estimators': [10,50,200],
               'max_features': ['auto', 'sqrt','log2'],
               'max_depth': [2,5],
               'min_samples_split' : [2,4],
               'min_samples_leaf' : [1,4],
               'bootstrap': [True, False]}, #rf
             {'n_estimators': [10,50,200],
              'learning_rate': np.linspace(0.01,1,10)}, #adaboost
             {'learning_rate': np.linspace(0.01,1,10),
              'n_estimators': [10,50,100],
              'max_features': ['auto', 'sqrt','log2'],
              'max_depth': [2,4,8],'min_samples_split' : [2,4]}, #grdboost
             {'C' : np.linspace(0.01,10,6),
              'gamma' : np.linspace(0.01,1,6),
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}] #svc

In [27]:
classification_models(tf_idf_data,df['Score'],grid_models,param_grid=param_grid,grid=True)

knn's best parameters are {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
tree's best parameters are {'criterion': 'gini', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 4}
rf's best parameters are {'bootstrap': False, 'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 10}
AdaBoost's best parameters are {'learning_rate': 0.56, 'n_estimators': 200}
GrdBoost's best parameters are {'learning_rate': 0.45, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 100}
svc's best parameters are {'C': 6.004, 'gamma': 0.6040000000000001, 'kernel': 'rbf'}


,Model,Accuracy,Recall,Precision,F1
0,knn,0.614927,0.997099,0.592529,0.743331
1,tree,0.625203,0.659574,0.666667,0.663102
2,rf,0.570579,0.978723,0.567265,0.718240
3,AdaBoost,0.674419,0.748549,0.693548,0.720000
4,GrdBoost,0.703624,0.794004,0.710208,0.749772
5,svc,0.775014,0.810445,0.792060,0.801147


It looks like our best model is SVC. Let's analyze this model's results further.

# Final Model